In [1]:
import pandas as pd
import matplotlib
import requests
import json


In [ ]:
%pip install geocoder

In [2]:
pd.set_option('display.max_rows', 300)

In [3]:
vdh_foodPermits = pd.read_csv('VDH_FoodPermits.csv')
main_data = vdh_foodPermits 
main_data

/var/folders/g6/vzn6mdrs5j3205mpgv3cwzfc0000gn/T/ipykernel_1205/2065872026.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  vdh_foodPermits = pd.read_csv('VDH_FoodPermits.csv')


,HealthDistrict,establishmentName,permitName,permitNumber,program,PermitType,status,RiskCategory,catering,commissary,...,inspectionFrequency,ApplicationDate,OriginalIssueDate,PermitIssueDate,ExpirationDate,FacilityAddress,city,zip,FIPSCode,LocalityName
0,EHD - Rappahannock,NaN,A-La-Carte by Weddings by Ginny,JCRS-5W9Q8L,Food Establishment,Restaurant/Caterer,Surrendered,1,NaN,NaN,...,NaN,2/12/04,6/18/04,6/18/04,6/30/05,NaN,NaN,NaN,NaN,NaN
1,EHD - Prince William,NaN,Chiyoshi,AJOS-ARKQPG,Food Establishment,Carry Out Food Service Only,Business Closed,3,NO,NO,...,NaN,9/22/17,3/29/18,4/1/19,3/31/22,NaN,NaN,NaN,NaN,NaN
2,EHD - Three Rivers,NaN,Monroe Bay Landing,JKAZ-59KLBD,Food Establishment,Restaurant,Surrendered,4,NaN,NaN,...,NaN,1/3/94,2/10/95,2/13/03,2/28/04,NaN,NaN,NaN,NaN,NaN
3,EHD - New River,NaN,Narrows High School-girls Basketball,ACOS-5NUP2H,Food Establishment,Temporary restaurant,Surrendered,NaN,NaN,NaN,...,NaN,6/25/03,6/25/03,7/4/03,7/4/03,NaN,NaN,NaN,NaN,NaN
4,EHD - Virginia Beach,NaN,No. One Chinese Restaurant,JLLD-5CLR4E,Food Establishment,Full Service Restaurant,Surrendered,4,NaN,NaN,...,NaN,7/1/03,7/8/03,7/1/08,7/31/09,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97044,EHD - Alleghany-Roanoke,509 E. Ridgeway Street,The Willow Tree,MCAH-86YFRX,Food Establishment,Fast Food Restaurant,Surrendered,2,NaN,NaN,...,NaN,6/30/10,7/9/10,7/9/10,7/9/11,509 E. Ridgeway Street,Clifton Forge,24422,5.0,CLIFTON FORGE
97045,EHD - Alleghany-Roanoke,117 Falcon Ridge Road,Bowman's Auction Co.,MCAH-85CGVE,Food Establishment,Mobile Food Unit,Surrendered,1,NaN,NaN,...,NaN,4/30/10,5/8/10,5/8/10,5/7/11,117 Falcon Ridge Road,Clifton Forge,24422,5.0,CLIFTON FORGE
97046,EHD - Cumberland Plateau,"Watson, Ray","Watson, Ray",MSTS-9WBLG4,Food Establishment,Other Food Service,Surrendered,NaN,NaN,NaN,...,NaN,8/5/15,NaN,NaN,NaN,431 Craig Street,Norton,24273,195.0,WISE
97047,EHD - Cumberland Plateau,Appalachain Shine II,Appalachain Shine II,KDEL-AMWS5B,Food Establishment,Mobile Food Unit,Expired,NaN,NO,NO,...,NaN,4/20/17,5/23/17,NaN,NaN,120 Ridge Ave,Appalachia,24216,195.0,WISE


In [4]:
clean_vdh = main_data.loc['HealthDistrict': 'RiskCategory']
clean_vdh['LocalityName'] = clean_vdh['LocalityName'].astype('string')

In [5]:
p_type = main_data['PermitType']
clean_cities = main_data['LocalityName']
newdf = pd.merge(p_type, clean_cities, left_index=True, right_index=True)

In [6]:
print(newdf)

                        PermitType   LocalityName
0               Restaurant/Caterer            NaN
1      Carry Out Food Service Only            NaN
2                       Restaurant            NaN
3             Temporary restaurant            NaN
4          Full Service Restaurant            NaN
...                            ...            ...
97044         Fast Food Restaurant  CLIFTON FORGE
97045             Mobile Food Unit  CLIFTON FORGE
97046           Other Food Service           WISE
97047             Mobile Food Unit           WISE
97048             Mobile Food Unit           WISE

[97049 rows x 2 columns]


## Cleaning the df to remove null counties and null permit types


In [7]:
p_type = vdh_foodPermits['PermitType']
clean_cities = vdh_foodPermits['LocalityName']
newdf = pd.merge(p_type, clean_cities, left_index=True, right_index=True)

In [8]:
clean_newdf = newdf[newdf['PermitType'].notna()]
merged_df = clean_newdf[clean_newdf['LocalityName'].notna()]
print(merged_df)

                 PermitType   LocalityName
9546       Mobile Food Unit   OUT OF STATE
9547       Mobile Food Unit   OUT OF STATE
9549       Mobile Food Unit   OUT OF STATE
9550       Mobile Food Unit   OUT OF STATE
9551                Caterer   OUT OF STATE
...                     ...            ...
97044  Fast Food Restaurant  CLIFTON FORGE
97045      Mobile Food Unit  CLIFTON FORGE
97046    Other Food Service           WISE
97047      Mobile Food Unit           WISE
97048      Mobile Food Unit           WISE

[76578 rows x 2 columns]


## Removing similar permit types 

In [9]:
# Replace values of 'Full Service Restaurant' with 'Full Service Restaurant/Caterer'
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Full Service Restaurant/Caterer','Full Service Restaurant' )
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Restaurant','Full Service Restaurant' )
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Restaurant/Caterer','Full Service Restaurant' )


# # Replace values of Mobile food unit with  Mobile Food Unit  

merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Mobile food unit', 'Mobile Food Unit') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Mobile Unit', 'Mobile Food Unit ') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Mobile Food Unit', 'Mobile Food Unit ')

# # Replace values of Carry out only   with Carry Out Food Service Only 

merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry out only' , 'Carry Out Food Service Only')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry Out Food Service Only ' , 'Carry Out Food Service Only')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry Out' , 'Carry Out Food Service Only')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry Out Only' , 'Carry Out Food Service Only')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry Out ' , 'Carry Out Food Service Only')

# # Replace values of Temporary restaurant  with Temporary Restaurant 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Temporary restaurant', 'Temporary Restaurant')

# # Replace values of Convenience Store Food Service  with Convenience Store  
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Convenience Store' , 'Convenience Store Food Service')

# # Replace values of Child Care Center with Child Care Food Service
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child Care Center', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child care', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child Care', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child Care With Food ', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Residential Child Care Institution Food Service', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child Care Kitchen', 'Child Care Food Service') 

# # Replace values of Summer Food Service Program Kitchen  with Summer Food Service Program Feeding Site 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Summer Food Service Program Kitchen', 'Summer Food Service Program Feeding Site')

# # Replace Nursing homes with Nursing Home Food Service
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Nursing homes', 'Nursing Home Food Service')

# # Replace Seasonal Full Service Restaurant with Seasonal restaurant
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal Full Service Restaurant' , 'Seasonal restaurant ')

# # Replace Seasonal restaurant  with Seasonal restaurant ( Gaps )  Seasonal Kitchen , Seasonal Restaurant
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal restaurant ', 'Seasonal Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal', 'Seasonal Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal Kitchen', 'Seasonal Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal restaurant', 'Seasonal Restaurant') 


# # # Replace Nursing homes with Adult Care Home Food Service  Adult Care/Hospital 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult Care Home', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult Home', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult day care', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult care homes', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult Day Care Food Service', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult Care/Hospital', 'Adult Care Home Food Service')
# Replace Group Home', 'Group Home Food Service'
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Group Home', 'Group Home Food Service')

# Other Service 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Other food service', 'Other Food Service')

# Jail Kitchen 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Jail kitchen', 'Jail Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Jail Food Service', 'Jail Food Service')


# # Replace Seasonal restaurant  with Fast Food Restaurant 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Fast Food Restaurant/Caterer', 'Fast Food Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Fast Food', 'Fast Food Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Fast Food Restaurant', 'Fast Food Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal Fast Food Restaurant', 'Fast Food Restaurant') 

# Private school kitchen  with Private School Kitchen
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Private school kitchen', 'Private School Kitchen') 





# Convinience store   Convenience store , 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Convenience store', 'Convenience Store Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Local Convenience Store', 'Convenience Store Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Convenience Store (Local)', 'Convenience Store Food Service')


# Private college food service
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Private college food service', 'Private College Food Service')


#summer camp 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Summer camp kitchen', 'Summer Camp Food Service')




# # Bed and Brekfast food service 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Bed & Breakfast', 'Bed & Breakfast Food Service') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Bed & Breakfast food service', 'Bed & Breakfast Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Continental Breakfast', 'Bed & Breakfast Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Hotel Continental Breakfast', 'Bed & Breakfast Food Service')

#Hospital Kitchcen 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Hospital kitchen', 'Hospital Food Service')


# Replace Seasonal restaurant  with Fast Food Restaurant 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Vending Machines', 'Vending Machine')

# Replace Grocery stores
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery Store (Bakery Dept.)', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery Store (Grocery Dept.)', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery Store (Meat/Poultry Dept.)', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery Store Food Service', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Local Grocery Store Food Service', 'Grocery Store') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Local Grocery Store', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery store', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Small Grocery Store ', 'Grocery Store')


/var/folders/g6/vzn6mdrs5j3205mpgv3cwzfc0000gn/T/ipykernel_1205/1106893685.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Full Service Restaurant/Caterer','Full Service Restaurant' )
/var/folders/g6/vzn6mdrs5j3205mpgv3cwzfc0000gn/T/ipykernel_1205/1106893685.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Restaurant','Full Service Restaurant' )
/var/folders/g6/vzn6mdrs5j3205mpgv3cwzfc0000gn/T/ipykernel_1205/1106893685.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [10]:

print(merged_df)
type(merged_df)

                    PermitType   LocalityName
9546          Mobile Food Unit   OUT OF STATE
9547          Mobile Food Unit   OUT OF STATE
9549          Mobile Food Unit   OUT OF STATE
9550          Mobile Food Unit   OUT OF STATE
9551                   Caterer   OUT OF STATE
...                        ...            ...
97045         Mobile Food Unit  CLIFTON FORGE
97046       Other Food Service           WISE
97047         Mobile Food Unit           WISE
97048         Mobile Food Unit           WISE
PermitType                 NaN            NaN

[76579 rows x 2 columns]


pandas.core.frame.DataFrame

## Total permit types

In [23]:
totalDf = merged_df.LocalityName.value_counts().reset_index(name='PermitType') #.iloc[:,1:2]
totalDf

,index,PermitType
0,FAIRFAX COUNTY,9808
1,VIRGINIA BEACH,5705
2,ARLINGTON,3842
3,HENRICO,3489
4,NORFOLK,3209
5,NEWPORT NEWS,2543
6,ALEXANDRIA,2404
7,CHESTERFIELD,2299
8,LOUDOUN,2024
9,CHESAPEAKE,1785


In [18]:
pd.set_option('display.max_rows', 200)

In [19]:
display(merged_df)

,PermitType,LocalityName
9546,Mobile Food Unit,OUT OF STATE
9547,Mobile Food Unit,OUT OF STATE
9549,Mobile Food Unit,OUT OF STATE
9550,Mobile Food Unit,OUT OF STATE
9551,Caterer,OUT OF STATE
...,...,...
97045,Mobile Food Unit,CLIFTON FORGE
97046,Other Food Service,WISE
97047,Mobile Food Unit,WISE
97048,Mobile Food Unit,WISE


## Group by the counties and permit types

In [20]:
result2 = merged_df.groupby(['LocalityName', 'PermitType']).size().reset_index(name='Count')
(result2)


,LocalityName,PermitType,Count
0,ACCOMACK,Adult Care Home Food Service,1
1,ACCOMACK,Adult Day Care Food Service,2
2,ACCOMACK,Bed & Breakfast Food Service,1
3,ACCOMACK,Carry Out Food Service Only,10
4,ACCOMACK,Caterer,2
...,...,...,...
3568,YORK,Public Middle or High School Food Service,7
3569,YORK,Restaurant,24
3570,YORK,Seasonal Fast Food Restaurant,22
3571,YORK,Seasonal Full Service Restaurant,2


In [ ]:
result2.to_excel("output2.xlsx") 

In [ ]:
print(result2["PermitType"].unique())
